# Key Consideration for Your Dataproc Cluster

1. Cluster Resources:mm

.Master: n2-standard-4 (4 vCPUs, 16 GB RAM, 32GB disk)
.Workers(2x): n2-standard-4 (4 vCPUs, 16 GB RAM , 64 GB disk each)
.Total: 8 worker vCPUs, ~32 GB RAM (excluding master node)

2. Dataproc Features Disabled:

.No autoscaling, Metastore, advanced execution layer, advanced optimizations
.Storage: pd-balanced (no SSDs, so I/O optimization is crucial)
.Networing: Internal IP enabled

3. Optimization Strategy:

.Tune shuffle partitions, broadcast join threshold, and storage persistence
.Adjust parallelism based on 2 workersx4 cores
.Avoid execessive caching due to disk-based storage

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
.appName('Olist Ecommerce Performance Optimization')\
.config('spark.executor.memory','6g')\
.config('spark.executor.cores','4')\
.config('spark.executor.instances','2')\
.config('spark.driver.memory','4g')\
.config('spark.driver.maxResultSize','2g')\
.config('spark.sql.shuffle.partitions','64')\
.config('spark.default.parallelism','64')\
.config('spark.sql.adaptive.enabled','true')\
.config('spark.sql.adaptive.coalescePartition.enabled','true')\
.config('spark.sql.autoBroadcatJoinThreshold',20*1024*1024)\
.config('spark.sql.files.maxPartitionBytes','64MB')\
.config('spark.sql.files.openCostInBytes','2MB')\
.config('spark.memory.fraction',0.8)\
.config('spark.memory.storageFraction',0.2)\
.getOrCreate()

25/05/15 03:32:06 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


⚙️ Cluster Summary
💡 Hardware Setup
Master: n2-standard-4 (4 vCPU, 16 GB RAM)

Workers (2×): n2-standard-4 (8 vCPUs total, ~32 GB RAM)

Disk: pd-balanced (HDD-like, not SSD — so disk I/O is a performance bottleneck)

🔧 Spark Configuration Explained

spark = SparkSession.builder\


🧠 Memory & Executor Settings

.config('spark.executor.memory','6g')
.config('spark.executor.cores','4')
.config('spark.executor.instances','2')
executor.memory=6g: Each executor gets 6 GB RAM.

executor.cores=4: Each executor uses all 4 cores of a worker node (since you have 2 workers with 4 cores each).

executor.instances=2: One executor per worker. This aligns with your cluster (2 workers × 4 cores).

✅ Best practice: Assign one executor per node to fully utilize resources without contention.

🧠 Driver Settings

.config('spark.driver.memory','4g')
.config('spark.driver.maxResultSize','2g')
driver.memory=4g: Driver runs on the master node (16 GB available).

maxResultSize=2g: Prevents massive result sets from overwhelming the driver.

🔄 Shuffle & Parallelism Tuning

.config('spark.sql.shuffle.partitions','64')
.config('spark.default.parallelism','64')
shuffle.partitions=64: Controls number of output partitions after joins/groupBy.

default.parallelism=64: Default number of tasks. Good for 8 cores × 2 tasks/core = 16–64 range.

💡 For your 8-core cluster, 64 is a safe upper bound to avoid under-parallelization.

⚙️ Adaptive Query Execution (AQE)

.config('spark.sql.adaptive.enabled','true')
.config('spark.sql.adaptive.coalescePartition.enabled','true')
AQE: Dynamically adjusts partition count, avoids skew, optimizes join strategies at runtime.

Coalesce enabled: Reduces small partitions after shuffles → better performance.

✅ This is crucial when you don’t know data sizes in advance.

🔁 Broadcast Join Threshold

.config('spark.sql.autoBroadcatJoinThreshold',20*1024*1024)
20 MB threshold: Tables smaller than 20 MB will be broadcast in joins, saving expensive shuffles.

✅ Useful when joining dimension tables (e.g., customers, sellers) with large fact tables.

📂 File I/O Tuning (especially important due to no SSD)

.config('spark.sql.files.maxPartitionBytes','64MB')
.config('spark.sql.files.openCostInBytes','2MB')
64 MB partition size: Avoids over-partitioning and excessive file reads.

2 MB open cost: Helps Spark optimize I/O operations by assigning fewer files per task if they are expensive to open.

✅ This compensates for your pd-balanced disks (which are slower than SSDs).

💾 Memory Management

.config('spark..memory.fraction',0.8)       # Typo: extra dot
.config('spark.memory.storageFraction',0.2)
memory.fraction=0.8: 80% of executor memory is used for execution and storage (default is 0.6)

storageFraction=0.2: Of that, only 20% is for caching. Leaves more room for computation.


🧠 Summary of Your Optimization Strategy
Area	Strategy & Rationale
Executors	One per node with full resource use.
Driver	Conservative size to handle large operations safely.
Shuffle	Balanced partitioning (64) for better performance on 8-core setup.
AQE	Dynamic optimization based on runtime data size & skew.
Broadcast Joins	Efficient small table joins using threshold.
Storage Tuning	Optimal file partition size and reduced open cost to handle slow disk I/O.
Memory	Favor execution over caching since you're not using SSDs and caching is expensive.

In [3]:
full_order_df = spark.read.parquet('/data/olist_proc/full_order_df_joined.parquet')

In [4]:
hdfs_path = '/data/olist/'

In [5]:
customers_df = spark.read.csv(hdfs_path + 'olist_customers_dataset.csv',header=True,inferSchema=True)
geolocation_df = spark.read.csv(hdfs_path + 'olist_geolocation_dataset.csv',header=True,inferSchema=True)
order_items_df = spark.read.csv(hdfs_path + 'olist_order_items_dataset.csv',header=True,inferSchema=True)
payments_df = spark.read.csv(hdfs_path + 'olist_order_payments_dataset.csv',header=True,inferSchema=True)
reviews_df = spark.read.csv(hdfs_path + 'olist_order_reviews_dataset.csv',header=True,inferSchema=True)
orders_df = spark.read.csv(hdfs_path + 'olist_orders_dataset.csv',header=True,inferSchema=True)
products_df = spark.read.csv(hdfs_path + 'olist_products_dataset.csv',header=True,inferSchema=True)
sellers_df = spark.read.csv(hdfs_path + 'olist_sellers_dataset.csv',header=True,inferSchema=True)
category_translation_df = spark.read.csv(hdfs_path + 'product_category_name_translation.csv',header=True,inferSchema=True)

# Optimized Join Strategies

In [6]:
from pyspark.sql.functions import *

In [7]:
# Broadcast

customer_broadcast_df = broadcast(customers_df)

optimized_broadcast_join = full_order_df.join(customer_broadcast_df,'customer_id')

In [12]:
# Sort and Merge join

sorted_customers_df = customers_df.sortWithinPartitions('customer_id')
sorted_orders_df = full_order_df.sortWithinPartitions('customer_id')

optimized_merge_full_order_df = sorted_orders_df.join(sorted_customers_df,'customer_id')

In [13]:
# Bucket Join

bucketed_customers_df = customers_df.repartition(10,'customer_id')
bucketed_orders_df = full_order_df.repartition(10,'customer_id')

bucketed_join_df = bucketed_orders_df.join(bucketed_customers_df,'customer_id')

In [19]:
 # Skew join handling
    
skew_handled_join = full_order_df.join(customers_df,'customer_id')

In [17]:
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", "true")
spark.conf.set("spark.sql.adaptive.skewJoin.skewedPartitionThresholdInBytes", 64 * 1024 * 1024)  # default


In [1]:
spark.stop()